## Arxiv Mongo Import

This function takes a set of CERMXML files created by CERM and converts them into json files before adding them to the MongoDB.  A parse status is added to each document in order to define what was extracted from the pdf.

In [1]:
import json
import xmltodict
import os
from pymongo import MongoClient
from tqdm import tqdm

In [14]:
db = MongoClient().ds_documents
col = db.papers

prefix = 'arxiv/'
folders = os.listdir(prefix)
folders = [x for x in folders if len(x) ==2]
for folder in folders:
    print('# Now completing folder:' + folder)
    files = os.listdir(prefix+folder)
    files = [x for x in files if x[-7:] == 'cermxml']
    for af in tqdm(files):
        doc = {}
        parse_keys = set()
        fi = prefix + folder + '/' + af
        with open(fi) as fp:
            entry = fp.read()
        data = xmltodict.parse(entry)
        doc['source'] = 'arxiv'
        doc['category'] = folder
        doc['doc_name'] = af
        doc['fpath'] = fi
        doc['original'] = data
        doc['parse_status'] = 'no_parse'
        try:
            doc['title'] = data['article']['front']['article-meta']['title-group']['article-title']
            doc['authors'] = data['article']['front']['article-meta']['contrib-group']['contrib']
            doc['abstract'] = data['article']['front']['article-meta']['abstract']
            doc['references'] = data['article']['back']['ref-list']['ref']
            doc['body'] = data['article']['body']['sec']
            doc['parse_status'] = 'full_parse'
            doc['parse_keys'] = str({'title','authors','abstract','references','body'})
        except:
            if data['article']['front'].get('article-meta',{}):
                if data['article']['front']['article-meta'].get('abstract'):
                    doc['abstract'] = data['article']['front']['article-meta']['abstract']
                    parse_keys.add('abstract')
            if data['article']['back'].get('ref-list',{}):
                doc['references'] = data['article']['back']['ref-list']['ref']
                parse_keys.add('references')
            if  data['article']['front'].get('article-meta',{}):
                if data['article']['front']['article-meta'].get('contrib-group'):
                    doc['authors'] = data['article']['front']['article-meta']['contrib-group']
                    parse_keys.add('authors')
            if data['article']['front'].get('article-meta',{}):
                if data['article']['front']['article-meta'].get('title-group',{}):
                    if data['article']['front']['article-meta']['title-group'].get('article-title'):
                        doc['title'] = data['article']['front']['article-meta']['title-group']['article-title']
                        parse_keys.add('title')
            if data['article'].get('body',{}):
                doc['body'] = data['article']['body']['sec']
                parse_keys.add('body')
            else:
                doc['parse_status'] = 'no_body'
            if len({'abstract', 'body', 'references'} - parse_keys) == 0:
                doc['parse_status'] = 'partial_parse'
            doc['parse_keys'] = str(parse_keys)
        result = col.insert_one(doc)

  1%|          | 11/1922 [00:00<00:17, 109.49it/s]

# Now completing folder:IR


  1%|          | 7/1242 [00:00<00:19, 62.08it/s]

# Now completing folder:NA


  0%|          | 12/19457 [00:00<02:45, 117.53it/s]

...

  0%|          | 10/5275 [00:00<00:54, 96.56it/s]

# Now completing folder:SI


  0%|          | 10/6543 [00:00<01:14, 87.12it/s]

# Now completing folder:NI


100%|██████████| 6543/6543 [01:00<00:00, 108.47it/s]


In [15]:
col.count()

128418

In [19]:
t = col.find_one()